In [13]:
import sqlite3
import pandas as pd

In [14]:
def run_query(query):
    with sqlite3.connect('AreaOvitrap.db') as conn:
        return pd.read_sql(query,conn)

def run_command(command):
    with sqlite3.connect('AreaOvitrap.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(command)

def show_tables():        
    query = '''
        SELECT name,
               type
        From sqlite_master
        WHERE type IN ("type","view");
    '''
    return run_query(query)

def district_code_generator(area):
    num_range = len(district_only[district_only['area_id'] == area])
    for i in range(1,num_range+1):
        yield area + "D{:02d}".format(i)

def location_code_generator(district_id):
    num_range = len(locations[locations['District_id'] == district_id])
    for i in range(1,num_range+1):
        yield district_id + "{:02d}".format(i)
        
def match_district(area_districts):
    for index,value in enumerate(locations['Eng']):
        for key, item in area_districts.items():
            if value in item:
                locations.loc[index,'District'] = key
    return locations

In [15]:
data = pd.read_csv("Area_Ovitrap_Index_Jan2008-Jul2018.csv")

In [16]:
all_districts = {
    'HK':{
        'Central Western':{'Central and Admiralty','Sai Wan','Sheung Wan and Sai Ying Pun'},
        'Eastern':{'Chai Wan West','Shau Kei Wan & Sai Wan Ho','North Point'},
        'Southern':{'Aberdeen and Ap Lei Chau','Pokfulam','Deep Water Bay & Repulse Bay'},
        'Wanchai':{'Tin Hau','Wan Chai North','Happy Valley'}
         },
    
    'KL':{
        'Yau Tsim':{'Tsim Sha Tsui','Tsim Sha Tsui East'},
        'Mong Kok':{'Mong Kok'},
        'Sham Shui Po':{'Cheung Sha Wan','Lai Chi Kok','Sham Shui Po East'},
        'Kowloon City':{'Ho Man Tin','Kowloon City North','Hung Hom','Lok Fu West','Kai Tak North'},
        'Wong Tai Sin':{'Wong Tai Sin Central','Diamond Hill','Ngau Chi Wan'},
        'Kwun Tong':{'Kwun Tong Central','Lam Tin','Yau Tong','Kowloon Bay'}
        },
    
    'NT':{
        'Sai Kung':{'Tseung Kwan O South','Tseung Kwan O North','Sai Kung Town'},
        'Sha Tin':{'Tai Wai','Yuen Chau Kok','Ma On Shan','Lek Yuen','Wo Che'},
        'Tai Po':{'Tai Po'},
        'North':{'Fanling','Sheung Shui'},
        'Yuen Long':{'Tin Shui Wai','Yuen Kong','Yuen Long Town'},
        'Tuen Mun':{'Tuen Mun North','Tuen Mun South','Tuen Mun West','So Kwun Wat'},
        'Tsuen Wan':{'Tsuen Wan Town','Tsuen Wan West','Ma Wan','Sheung Kwai Chung'},
        'Kwai Tsing':{'Kwai Chung','Lai King','Tsing Yi North','Tsing Yi South'}
        },
    
    'IL':{
        'Islands':{'Cheung Chau','Tung Chung'}
        }
}


In [17]:
# matching the Chinese and English names of the districts into variable "translations"
chi_district = ['中西區','東區','南區','灣仔區','油尖區','旺角區','深水埗區','九龍城區','黃大仙區','觀塘區','西貢區','沙田區','大埔區','北區','元朗區','屯門區','荃灣區','葵青區','離島區']
eng_district = []
for area, district in all_districts.items():
    for key, _ in district.items():
        eng_district.append(key)
translations = list(zip(eng_district,chi_district))

In [18]:
# group the districts into their corresponding area
area_district = []
for area, district in all_districts.items():
    for key,value in district.items():
        area_district.append([area,key])

for index, value in enumerate(translations):
    area_district[index].append(value[1])


In [19]:
area_district

[['HK', 'Central Western', '中西區'],
 ['HK', 'Eastern', '東區'],
 ['HK', 'Southern', '南區'],
 ['HK', 'Wanchai', '灣仔區'],
 ['KL', 'Yau Tsim', '油尖區'],
 ['KL', 'Mong Kok', '旺角區'],
 ['KL', 'Sham Shui Po', '深水埗區'],
 ['KL', 'Kowloon City', '九龍城區'],
 ['KL', 'Wong Tai Sin', '黃大仙區'],
 ['KL', 'Kwun Tong', '觀塘區'],
 ['NT', 'Sai Kung', '西貢區'],
 ['NT', 'Sha Tin', '沙田區'],
 ['NT', 'Tai Po', '大埔區'],
 ['NT', 'North', '北區'],
 ['NT', 'Yuen Long', '元朗區'],
 ['NT', 'Tuen Mun', '屯門區'],
 ['NT', 'Tsuen Wan', '荃灣區'],
 ['NT', 'Kwai Tsing', '葵青區'],
 ['IL', 'Islands', '離島區']]

In [7]:
# create a pandas dataframe for the data of all districts
district_only = pd.DataFrame(area_district,columns=['area_id','eng_district','chi_district'])   
hk_code = district_code_generator('HK') # generate ID for main area "Hong Kong Island"
kl_code = district_code_generator('KL') # generate ID for main area "Kowloon"
nt_code = district_code_generator('NT') # generate ID for main area "New Territories"
il_code = district_code_generator('IL') # generate ID for main area "Islands"
district_code = [hk_code,kl_code,nt_code,il_code]

for index,value in enumerate(district_only['area_id']):
    for i, area in enumerate(['HK','KL','NT','IL']):
        if value == area:
            district_only.loc[index,'District_id'] = next(district_code[i])                    
         

In [8]:
cols = district_only.columns.tolist()
cols = cols[-1:]+cols[:-1]
district_only = district_only[cols]

In [9]:
area_dict = {'area_id':['HK','KL','IL','NT'],
                 'eng_area':['Hong Kong Island','Kowloon','Islands','New Territories'],
                 'chi_area':['香港島','九龍','離島','新界']}

t_area = '''
    CREATE TABLE IF NOT EXISTS area(
    area_id TEXT PRIMARY KEY,
    eng_area TEXT,
    chi_area TEXT
    )
'''

run_command(t_area)

area = pd.DataFrame(area_dict)
with sqlite3.connect("AreaOviTrap.db") as conn:
    area.to_sql('area',conn,if_exists='append',index=False)

In [10]:
run_query("SELECT * FROM area")

,area_id,eng_area,chi_area
0,HK,Hong Kong Island,香港島
1,KL,Kowloon,九龍
2,IL,Islands,離島
3,NT,New Territories,新界


In [11]:
t_district = '''
    CREATE TABLE IF NOT EXISTS district(
    district_id TEXT PRIMARY KEY,
    area_id TEXT,
    eng_district TEXT,
    chi_district TEXT,
    FOREIGN KEY (area_id) REFERENCES area(area_id)
    )
'''

run_command(t_district)
with sqlite3.connect("AreaOviTrap.db") as conn:
    district_only.to_sql('district',conn,if_exists='append',index=False)

In [12]:
run_query("SELECT * FROM district")

,district_id,area_id,eng_district,chi_district
0,HKD01,HK,Central Western,中西區
1,HKD02,HK,Eastern,東區
2,HKD03,HK,Southern,南區
3,HKD04,HK,Wanchai,灣仔區
4,KLD01,KL,Yau Tsim,油尖區
5,KLD02,KL,Mong Kok,旺角區
6,KLD03,KL,Sham Shui Po,深水埗區
7,KLD04,KL,Kowloon City,九龍城區
8,KLD05,KL,Wong Tai Sin,黃大仙區
9,KLD06,KL,Kwun Tong,觀塘區


In [9]:
# extracting unique location from the data
eng_loc = data['Eng'].unique()
chi_loc = data['Chi'].unique()
locations = pd.DataFrame({'District':'','Eng':eng_loc,'Chi':chi_loc})

hk_districts = all_districts['HK']
kl_districts = all_districts['KL']
nt_districts = all_districts['NT']
il_districts = all_districts['IL']
four_district = [hk_districts,kl_districts,nt_districts,il_districts]

# match the location with the correpsonding district
for each in four_district:
    locations = match_district(each)

# match the location with corresponding district_id
for index, value in enumerate(locations['District']):
    for i, district in enumerate(district_only['eng_district']):
        if value == district:
            locations.loc[index,'District_id'] = district_only.loc[i,'District_id']
            
# generate Location_id by using location_code_generator
unique_district_id = locations['District_id'].unique().tolist()   
for each in unique_district_id:
    code = location_code_generator(each)
    for index,value in enumerate(locations['District_id']):
        if value == each:
            locations.loc[index,'Location_id'] = next(code)                      

In [10]:
locations.head()

,District,Eng,Chi,District_id,Location_id
0,Eastern,Chai Wan West,柴灣西,HKD02,HKD0201
1,Wanchai,Wan Chai North,灣仔北,HKD04,HKD0401
2,Wanchai,Happy Valley,跑馬地,HKD04,HKD0402
3,Central Western,Sheung Wan and Sai Ying Pun,上環及西營盤,HKD01,HKD0101
4,Central Western,Sai Wan,西環,HKD01,HKD0102


In [11]:
for index,value in enumerate(data['Eng']):
    for i, name in enumerate(locations['Eng']):
        if value == name:
            data.loc[index,'District_id'] = locations.loc[i,'District_id']
            data.loc[index,'Location_id'] = locations.loc[i,'Location_id']
            
with sqlite3.connect('AreaOvitrap.db') as conn:
    data.to_sql('origin',conn,index=False)            

In [12]:
data.head(20)       

,Eng,Chi,Date,AOI,Classification,District_id,Location_id
0,Chai Wan West,柴灣西,01-2008,0.000,1,HKD02,HKD0201
1,Chai Wan West,柴灣西,02-2008,0.000,1,HKD02,HKD0201
2,Chai Wan West,柴灣西,03-2008,0.000,1,HKD02,HKD0201
3,Chai Wan West,柴灣西,04-2008,0.057,2,HKD02,HKD0201
4,Chai Wan West,柴灣西,05-2008,0.073,2,HKD02,HKD0201
5,Chai Wan West,柴灣西,06-2008,0.236,3,HKD02,HKD0201
6,Chai Wan West,柴灣西,07-2008,0.078,2,HKD02,HKD0201
7,Chai Wan West,柴灣西,08-2008,0.096,2,HKD02,HKD0201
8,Chai Wan West,柴灣西,09-2008,0.054,2,HKD02,HKD0201
9,Chai Wan West,柴灣西,10-2008,0.019,1,HKD02,HKD0201


In [17]:
with sqlite3.connect('AreaOvitrap.db') as conn:
    data.to_sql('origin',conn,index=False)

In [18]:
t_location = '''
    CREATE TABLE IF NOT EXISTS location(
    location_id TEXT PRIMARY KEY,
    eng_location TEXT,
    chi_location TEXT,
    district_id,
    FOREIGN KEY (district_id) REFERENCES district(district_id)
    )
'''
location_data = '''
    INSERT OR IGNORE INTO location
    SELECT
        DISTINCT Location_id,
        Eng,
        Chi,
        District_id
    FROM origin
'''
run_command(t_location)
run_command(location_data)

In [19]:
t_aoi = '''
    CREATE TABLE IF NOT EXISTS area_ovitrap_index(
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    location_id TEXT,
    date TEXT,
    AOI FLOAT,
    Classification INTEGER,
    FOREIGN KEY (location_id) REFERENCES location(location_id)
    )
'''
aoi_data = '''
    INSERT OR IGNORE INTO area_ovitrap_index (location_id, date, AOI, Classification)
    SELECT
        Location_id,
        DATE,
        AOI,
        Classification
    FROM origin
'''
run_command(t_aoi)
run_command(aoi_data)

In [20]:
run_command("DROP TABLE IF EXISTS origin")
show_tables()

,name,type
0,area,table
1,sqlite_autoindex_area_1,index
2,district,table
3,sqlite_autoindex_district_1,index
4,location,table
5,sqlite_autoindex_location_1,index
6,area_ovitrap_index,table
7,sqlite_sequence,table
